<center><h1><b>ĐỒ ÁN: EEDI - MINING MISCONCEPTIONS IN MATHEMATICS</b></h1><center>

<center><h2><u>Môn:</u> Học sâu cho Khoa học dữ liệu - <u>Lớp:</u> 21_21</h2></center>

<h3><u>Giáo viên hướng dẫn:</u><h3>

- TS. Nguyễn Tiến Huy
- TS. Lê Thanh Tùng
- ThS. Nguyễn Trần Duy Minh  

<h3><u>Nhóm thực hiện:</u> Nhóm 1</h3>

| Họ và tên | MSSV |
| :--- | :---:|
| Phạm Nhật Duy | 21120058 |
| Nguyễn Trúc Nguyên | 21120102 |
| Trương Công Trung | 21120158 |
| Lê Trần Minh Khuê | 21120279 |

# **Tổng quan**

<u>Trong phạm vi đồ án này</u>, nhóm chúng em sẽ phát triển một mô hình học máy cung cấp khả năng phân tích và dự đoán mối liên hệ giữa *các ngộ nhận* với *các câu trả lời không chính xác* trong các câu hỏi trắc nghiệm.

<u>Mục tiêu:</u> Đối với mỗi câu hỏi trắc nghiệm, mô hình sẽ

- Xác định được câu trả lời đúng,
- Xác định được những lầm tưởng/ ngộ nhận có thể dẫn đến việc chọn đáp án sai.

<u>Trong phạm vi file này</u>, nhóm sẽ tiến hành xây dựng quy trình áp dụng mô hình ngôn ngữ lớn để xác định những ngộ nhận có thể dẫn đến việc chọn đáp án sai từ bộ dữ liệu Eedi.


# **First retrieval - Retrieve**

In [ ]:
!pip install /kaggle/input/k/justdylan/eedi-library/peft-0.14.0-py3-none-any.whl  --no-index --find-links=/kaggle/input/eedi-library --quiet

In [ ]:
!pip install /kaggle/input/eedi-library-from-sinchiro/autoawq-0.2.7.post1-py3-none-any.whl  --no-index --find-links=/kaggle/input/eedi-library-from-sinchiro --quiet

In [ ]:
import os, math, numpy as np
import sys
from transformers import AutoTokenizer
import pandas as pd
from tqdm import tqdm
import re, gc
import torch
pd.set_option('display.max_rows', 300)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
IS_SUBMISSION = True

MISCONCEPTION_MAPPING_PATH = "/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv"
TRAIN_DATA_PATH = "/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv"
TEST_DATA_PATH = "/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv"

LORA_PATH = '/kaggle/input/2211-lora-14b/transformers/default/1'
MODEL_PATH = "/kaggle/input/qw14b-awq/transformers/default/1"

In [ ]:
print('IS_SUBMISSION:', IS_SUBMISSION)

df_misconception_mapping = pd.read_csv(MISCONCEPTION_MAPPING_PATH)

if not IS_SUBMISSION:
    df_ret = pd.read_csv(TRAIN_DATA_PATH).fillna(-1).sample(100, random_state=103).reset_index(drop=True)
else:
    df_ret = pd.read_csv(TEST_DATA_PATH)

df_ret

IS_SUBMISSION: True


,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText
0,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets
1,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify
2,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct


+ Đầu tiện cần định dạng dữ liệu bằng cách tạo ra một cấu trúc câu hỏi chuẩn hóa với đầy đủ thông tin, bao gồm:
  + Nội dung câu hỏi.
  + Câu trả lời đúng.
  + Câu trả lời sai mà học sinh chọn.
+ Sau đó tạo một chuỗi đầu vào (prompt) cho mô hình ngôn ngữ lớn để phân tích và dự đoán.

In [ ]:
def format_input(row, wrong_choice):
    assert wrong_choice in "ABCD"
    # Extract values from the row
    question_text = row.get("QuestionText", "No question text provided")
    subject_name = row.get("SubjectName", "Unknown subject")
    construct_name = row.get("ConstructName", "Unknown construct")

    # Extract the correct and wrong answer text based on the choice
    correct_answer = row.get("CorrectAnswer", "Unknown")
    assert wrong_choice != correct_answer
    correct_answer_text = row.get(f"Answer{correct_answer}Text", "No correct answer text available")
    wrong_answer_text = row.get(f"Answer{wrong_choice}Text", "No wrong answer text available")

    # Construct the question format
    formatted_question = f"""Question: {question_text}

SubjectName: {subject_name}
ConstructName: {construct_name}"""

    TEMPLATE_INPUT = '{QUESTION}\nCorrect answer: {CORRECT_ANSWER}\nStudent wrong answer: {STUDENT_WRONG_ANSWER}'

    # Return the extracted data
    ret = {
        "QUESTION": formatted_question,
        "CORRECT_ANSWER": correct_answer_text,
        "STUDENT_WRONG_ANSWER": wrong_answer_text,
        "MISCONCEPTION_ID": row.get('Misconception{wrong_choice}Id'),
    }
    ret["PROMPT"] = TEMPLATE_INPUT.format(**ret)

    return ret

In [ ]:
items = []
rows = []
for _, row in df_ret.iterrows():
    for choice in ['A', 'B', 'C', 'D']:
        if choice == row["CorrectAnswer"]:
            continue

        correct_answer = row[f"Answer{row['CorrectAnswer']}Text"]

        item = {
            'QuestionId_Answer': f"{row['QuestionId']}_{choice}",
            'Prompt': format_input(row, choice)['PROMPT']
        }
        items.append(item)

        query_text = (f"Subject name: {row['SubjectName']}\n"
                      f"Construct name: {row['ConstructName']}\n"
                      f"Question: {row['QuestionText']}\n"
                      f"Correct answer: {correct_answer}\n"
                      f"### Misconcepte incorrect answer: {choice}.{row[f'Answer{choice}Text']}")

        rows.append({
            "query_text": query_text,
            "QuestionId_Answer": f"{row['QuestionId']}_{choice}",
            "ConstructName": row["ConstructName"],
            "SubjectName": row["SubjectName"],
            "QuestionText": row["QuestionText"],
            "correct_answer": correct_answer,
            "incorrect_answer": row[f"Answer{choice}Text"]
        })

df_input = pd.DataFrame(items)
df = pd.DataFrame(rows)

+ Tạo các truy vấn chi tiết từ dữ liệu đầu vào. Sau đó, xử lý truy vấn để tạo nhúng (`embeddings`) bằng cách chuẩn hóa và định dạng lại truy vấn để phù hợp với mô hình ngôn ngữ lớn bao gồm cắt ngắn nếu vượt quá độ dài tối đa (`query_max_len`) và thêm tiền tố/hậu tố vào truy vấn.

In [ ]:
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'<instruct>{task_description}\n<query>{query}'

def get_new_queries(queries, query_max_len, examples_prefix, tokenizer):
    inputs = tokenizer(
        queries,
        max_length=query_max_len - len(tokenizer('<s>', add_special_tokens=False)['input_ids']) - len(
            tokenizer('\n<response></s>', add_special_tokens=False)['input_ids']),
        return_token_type_ids=False,
        truncation=True,
        return_tensors=None,
        add_special_tokens=False
    )
    prefix_ids = tokenizer(examples_prefix, add_special_tokens=False)['input_ids']
    suffix_ids = tokenizer('\n<response>', add_special_tokens=False)['input_ids']
    new_max_length = (len(prefix_ids) + len(suffix_ids) + query_max_len + 8) // 8 * 8 + 8
    new_queries = tokenizer.batch_decode(inputs['input_ids'])
    for i in range(len(new_queries)):
        new_queries[i] = examples_prefix + new_queries[i] + '\n<response>'
    return new_max_length, new_queries

task =  "Given a math multiple-choice problem with a student's wrong answer, retrieve the math misconceptions"
queries = [
    get_detailed_instruct(task, q) for q in df_input['Prompt']
]
documents = df_misconception_mapping['MisconceptionName'].tolist()
query_max_len, doc_max_len = 320, 48
tokenizer = AutoTokenizer.from_pretrained(LORA_PATH)
examples_prefix = ''
new_query_max_len, new_queries = get_new_queries(queries, query_max_len, examples_prefix, tokenizer)

import json
with open('data.json', 'w') as f:
    data = {'texts': new_queries + documents}
    f.write(json.dumps(data))

+ Tiếp theo tạo vector embedding cho các đoạn văn bản đầu vào sử dụng mô hình ngôn ngữ lớn.
+ Bên cạnh đó áp dụng kỹ thuật LoRA (Low-Rank Adaptation) để giảm tải bộ nhớ khi huấn luyện hoặc inferencing với mô hình lớn.

In [ ]:
%%writefile run_embed.py
import argparse
import os
import json
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import peft

MAX_LENGTH = 320

def last_token_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    left_padding = attention_mask[:, -1].sum() == attention_mask.shape[0]
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[
            torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths
        ]

def get_embeddings_in_batches(model, tokenizer, texts, max_length, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
        batch_texts = texts[i : i + batch_size]
        batch_dict = tokenizer(
            batch_texts,
            max_length=max_length,
            padding=True,
            truncation=True,
            return_tensors="pt",
        ).to("cuda")
        with torch.no_grad(), torch.amp.autocast("cuda"):
            outputs = model(**batch_dict)
            batch_embeddings = last_token_pool(
                outputs.last_hidden_state, batch_dict["attention_mask"]
            )
            batch_embeddings = F.normalize(batch_embeddings, p=2, dim=1).cpu()
        embeddings.append(batch_embeddings)
    return torch.cat(embeddings, dim=0)

def load_model_and_tokenizer(base_model_path, lora_path, load_in_4bit=True):
    model = AutoModel.from_pretrained(
        base_model_path,
        device_map=0,
        torch_dtype=torch.float16,
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        lora_path if lora_path else base_model_path
    )
    model.resize_token_embeddings(len(tokenizer))
    if lora_path:
        model = peft.PeftModel.from_pretrained(model, lora_path)
    return model, tokenizer

def main(args):
    output_file = args.input_text.replace(
        ".json", ".pt.fold.{}.{}.embed".format(*args.fold)
    )
    if os.path.exists(output_file):
        print(f"Output file {output_file} already exists. Skipping...")
        return
    model, tokenizer = load_model_and_tokenizer(
        args.base_model, args.lora_path, load_in_4bit=args.load_in_4bit
    )
    texts = json.load(open(args.input_text))["texts"][args.fold[0] :: args.fold[1]]
    embeddings = get_embeddings_in_batches(
        model,
        tokenizer,
        texts,
        max_length=MAX_LENGTH,
        batch_size=4,
    )
    text2embeds = {text: emb for text, emb in zip(texts, embeddings)}
    torch.save(text2embeds, output_file)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--base_model",
        type=str,
        default="Qwen/Qwen2.5-14B",
        help="Path to the base model",
    )
    parser.add_argument(
        "--lora_path",
        type=str,
        default=None,
        help="Path to the LoRA model",
    )
    parser.add_argument(
        "--input_text",
        type=str,
        default=".cache/data.json",
    )
    parser.add_argument(
        "--load_in_4bit",
        action="store_true",
        help="Load model in 4-bit mode",
    )
    parser.add_argument("--fold", nargs=2, type=int, default=[0, 1])
    args = parser.parse_args()
    if not os.path.exists(args.lora_path):
        args.lora_path = None
    main(args)

Writing run_embed.py


+ Xử lý dữ liệu theo từng phân đoạn để tối ưu bộ nhớ và hiệu năng.

In [ ]:
cmd = f"(CUDA_VISIBLE_DEVICES=0 python run_embed.py --base_model {MODEL_PATH} --lora_path {LORA_PATH} --input_text data.json --fold 0 2) & (CUDA_VISIBLE_DEVICES=1 python run_embed.py --base_model {MODEL_PATH} --lora_path {LORA_PATH} --input_text data.json --fold 1 2)"
os.system(cmd)

ERROR! Intel® Extension for PyTorch* needs to work with PyTorch 2.5.*, but PyTorch 2.4.0 is found. Please switch to the matching version and run again.
ERROR! Intel® Extension for PyTorch* needs to work with PyTorch 2.5.*, but PyTorch 2.4.0 is found. Please switch to the matching version and run again.
ERROR! Intel® Extension for PyTorch* needs to work with PyTorch 2.5.*, but PyTorch 2.4.0 is found. Please switch to the matching version and run again.
ERROR! Intel® Extension for PyTorch* needs to work with PyTorch 2.5.*, but PyTorch 2.4.0 is found. Please switch to the matching version and run again.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/accelerate/utils/imports.py:336: UserWarning: Intel Extension for PyTorch 2.5 needs to work with PyTorch 2.5.*, but PyTorch 2.4.0 is found. Please switch to the matching version and run again.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/utils/imports.py:336: UserWarning: Intel Extension for PyTorch 2.5 needs to work with PyTorch 2.5.*, but PyTorch 2.4.0 is found. Please switch to the matching version and run again.
  warnings.warn(
Embedding: 100%|██████████| 325/325 [03:25<00:00,  1.59it/s]


0

+ Cần đảm bảo rằng tất cả các tệp nhúng đã được tạo xong và hợp nhất chúng vào một cấu trúc duy nhất (`text_to_embed`).

In [ ]:
from glob import glob
import time
text_to_embed = {}
files = glob('*.pt*')
while len(files) != 2:
    time.sleep(1)
    files = glob('*.pt*')

time.sleep(3)
for path in files:
    print(path)
    text_to_embed.update(torch.load(path))

data.pt.fold.0.2.embed
data.pt.fold.1.2.embed


/tmp/ipykernel_30/3571780413.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  text_to_embed.update(torch.load(path))


+ Sử dụng tích vô hướng giữa các vector nhúng để xác định độ tương đồng giữa sai lầm trong câu trả lời (truy vấn) và các ngộ nhận đã có (tài liệu).
+ Sau đó lấy danh sách của 25 ngộ nhận phù hợp nhất cho mỗi truy vấn, theo thứ tự giảm dần của điểm tương đồng.

In [ ]:
query_embeddings = torch.stack([text_to_embed[t] for t in new_queries])
doc_embeddings = torch.stack([text_to_embed[t] for t in documents])
query_embeddings.shape, doc_embeddings.shape

(torch.Size([9, 5120]), torch.Size([2587, 5120]))

In [ ]:
scores = query_embeddings @ doc_embeddings.T  # Shape: (M, N)
sorted_indices = torch.argsort(scores,1, descending=True)[:,:25].tolist()

+ Để dữ liệu đã được xử lý có thể được sử dụng lại mà không cần thực hiện lại toàn bộ pipeline, ta lưu lại kết quả vào hai tệp để sử dụng trong các bước tiếp theo.

In [ ]:
np.save("indices.npy", sorted_indices)
df.to_parquet("df.parquet", index=False)

In [ ]:
torch.cuda.empty_cache()

---

# **Second retrieval - Rerank**

In [ ]:
!pip uninstall -y torch --quiet
!pip install -q --no-index --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-vllm vllm --quiet
!pip install -q -U /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --quiet
!pip install -q -U /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl --quiet
!pip install -q --no-deps --no-index /kaggle/input/hf-libraries/sentence-transformers/sentence_transformers-3.1.0-py3-none-any.whl --quiet
!pip install --no-deps --no-index /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl --quiet

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
!pip install transformers peft accelerate \
    -q -U --no-index --find-links /kaggle/input/lmsys-wheel-files --quiet

In [ ]:
%%capture
!pip install --no-index /kaggle/input/bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
!pip install --no-index  /kaggle/input/bitsandbytes0-42-0/optimum-1.21.2-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
!pip install --no-index  /kaggle/input/bitsandbytes0-42-0/auto_gptq-0.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --find-links=/kaggle/input/bitsandbytes0-42-0

+ Tạo prompt cung cấp đầy đủ ngữ cảnh cho mô hình để phân tích và chọn ngộ nhận phù hợp.
    + Mô tả ngữ cảnh (`ConstructName` và `SubjectName`).
    + Thông tin về câu hỏi, đáp án đúng, và đáp án sai.
    + Danh sách các ngộ nhận (`Retrieval`).
  
+ Tải mô hình ngôn ngữ lớn (với cấu hình `awq` và `16-bit floating-point` để tối ưu cho GPU):
    + Xếp hạng các ngộ nhận qua từng vòng lặp.
    + Dùng LLM để dự đoán ngộ nhận phù hợp nhất cho từng câu hỏi.

In [ ]:
%%writefile run_vllm.py

import vllm
import numpy as np
import pandas as pd
from transformers import PreTrainedTokenizer, AutoTokenizer
from typing import List
import torch
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
import re

model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"
tokenizer = AutoTokenizer.from_pretrained(model_path)

def preprocess_text(x):
    x = re.sub("http\w+", '',x)
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = x.strip()
    return x

PROMPT  = """Here is a question about {ConstructName}({SubjectName}).
Question: {Question}
Correct Answer: {CorrectAnswer}
Incorrect Answer: {IncorrectAnswer}

You are a Mathematics teacher. Your task is to reason and identify the misconception behind the Incorrect Answer with the Question.
Answer concisely what misconception it is to lead to getting the Incorrect Answer.
Pick the correct misconception number from the below:

{Retrieval}
"""

def apply_template(row, tokenizer):
    messages = [
        {
            "role": "user",
            "content": preprocess_text(
                PROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"incorrect_answer"],
                    CorrectAnswer=row[f"correct_answer"],
                    Retrieval=row[f"retrieval"]
                )
            )
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text

df = pd.read_parquet("df.parquet")
indices = np.load("indices.npy")
df_misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

llm = vllm.LLM(
    model_path,
    quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.98,
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True
)
tokenizer = llm.get_tokenizer()

def get_candidates(c_indices):
    candidates = []

    mis_names = df_misconception_mapping["MisconceptionName"].values
    for ix in c_indices:
        c_names = []
        for i, name in enumerate(mis_names[ix]):
            c_names.append(f"{i+1}. {name}")

        candidates.append("\n".join(c_names))

    return candidates

survivors = indices[:, -1:]

for i in range(3):
    c_indices = np.concatenate([indices[:, -8*(i+1)-1:-8*i-1], survivors], axis=1)

    df["retrieval"] = get_candidates(c_indices)
    df["text"] = df.apply(lambda row: apply_template(row, tokenizer), axis=1)

    print("Example:")
    print(df["text"].values[0])
    print()

    responses = llm.generate(
        df["text"].values,
        vllm.SamplingParams(
            n=1,
            top_k=1,
            temperature=0,
            seed=777,
            skip_special_tokens=False,
            max_tokens=1,
            logits_processors=[MultipleChoiceLogitsProcessor(tokenizer, choices=["1", "2", "3", "4", "5", "6", "7", "8", "9"])]
        ),
        use_tqdm=True
    )

    responses = [x.outputs[0].text for x in responses]
    df["response"] = responses

    llm_choices = df["response"].astype(int).values - 1

    survivors = np.array([cix[best] for best, cix in zip(llm_choices, c_indices)]).reshape(-1, 1)

results = []

for i in range(indices.shape[0]):
    ix = indices[i]
    llm_choice = survivors[i, 0]

    results.append(" ".join([str(llm_choice)] + [str(x) for x in ix if x != llm_choice]))

df["MisconceptionId"] = results
df.to_csv("submission.csv", columns=["QuestionId_Answer", "MisconceptionId"], index=False)

Writing run_vllm.py


In [ ]:
!python run_vllm.py

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
WARNING 01-14 17:28:04 config.py:246] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 01-14 17:28:04 config.py:715] Defaulting to use mp for distributed inference
INFO 01-14 17:28:04 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1', speculative_config=None, tokenizer='/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=5120, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_con

In [ ]:
pd.read_csv("submission.csv")

,QuestionId_Answer,MisconceptionId
0,1869_B,1345 706 1507 2306 328 1672 1005 2518 1963 253...
1,1869_C,1345 2306 1507 706 1005 1999 2488 2532 987 251...
2,1869_D,315 1005 328 1507 2532 1672 1516 706 1345 2306...
3,1870_A,891 2142 2068 167 418 1755 979 113 1421 320 22...
4,1870_B,891 2142 2068 167 341 979 1755 1871 143 418 11...
5,1870_C,891 2142 2068 167 1755 418 113 2078 143 979 26...
6,1871_A,1287 1073 2439 1665 2551 1306 1059 1098 1677 1...
7,1871_C,1287 1073 2439 1665 2551 1098 1059 912 1306 16...
8,1871_D,1287 1073 1059 1866 903 2471 912 2439 2064 167...


---

# **Evaluation**

+ Tính `average precision` cho một cặp nhãn thực tế và dự đoán với giới hạn top-k kết quả.
+ Từ đó xác định `mean average precision` (độ chính xác trung bình tổng thể) cho toàn bộ danh sách nhãn thực tế và dự đoán.

In [ ]:
import numpy as np

def apk(actual, predicted, k=25):
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=25):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
if not IS_SUBMISSION:
    predicted = pd.read_csv("submission.csv")["MisconceptionId"].apply(lambda x: [int(y) for y in x.split()])

    df_label = {}

    for _, row in tqdm(df_ret.iterrows(), total=len(df_ret)):
        for option in ["A", "B", "C", "D"]:
            if (row["CorrectAnswer"] != option) and (row[f"Misconception{option}Id"] != -1):
                df_label[f"{row['QuestionId']}_{option}"] = [row[f"Misconception{option}Id"]]

    label = df_label.values()

    print("Validation: ", mapk(label, predicted))